In [54]:
from selenium import webdriver as wd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지기동 전 대기시간
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

import xlsxwriter as xw

from io import BytesIO
import urllib.request as req

from PIL import Image

! pip install PIL

def search(param):
    chrome_options = Options()

    # Headless 모드 : 브라우저를 안 띄우고 사용하는 모드
    chrome_options.add_argument('--headless')

    # webdriver 설정(Chrome, Firefox...) - Headless 모드로 설정됨
    browser = wd.Chrome('./webdriver/chromedriver.exe', options=chrome_options)

    # 일반모드(브라우저 띄움)
    # browser = wd.Chrome('./webdriver/chromedriver.exe')

    # 엑셀 처리하기 위한 workbook 생성
    workbook = xw.Workbook('./test11111.xlsx')

    # 워크시트 생성
    worksheet = workbook.add_worksheet()

    # 브라우저 내부 대기시간
    # 랜더링하는 5초안에 브라우저가 안 띄워지면 에러발생
    browser.implicitly_wait(5)

    # 페이지 이동
    browser.get('https://www.op.gg/')

    # 검색어 입력박스 선택
    element = browser.find_element_by_css_selector(\
            'form.summoner-search-form > input.summoner-search-form__text._suggest')

    # 검색어 입력
    element.send_keys(param)

    # 검색어 전송(submit)
    element.submit()

    time.sleep(4)

    # 엑셀의 행번호
    row_num = 1

    # 솔랭 정보만 가져옴
    WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, \
                '//*[@id="right_gametype_soloranked"]/a'))).click()
    
    time.sleep(6)
    
#     print(f'page content : {browser.page_source}')
    
    #     # bs4 초기화
    bs = BeautifulSoup(browser.page_source, 'lxml')

    item_list = bs.select('div.RealContent div.Content > div.GameItemList > div')
    # print(len(item_list))

    for item in item_list:
        game_type = item.select('div.GameType')[0].text.strip()
        game_result = item.select('div.GameResult')[0].text.strip()
        champion_name = item.select('div.ChampionName > a')[0].text.strip()
        kill = str(item.select('div.Content > div.KDA > div.KDA > .Kill')[0].text.strip())
        death = str(item.select('div.Content > div.KDA > div.KDA > .Death')[0].text.strip())
        assist = str(item.select('div.Content > div.KDA > div.KDA > .Assist')[0].text.strip())
        kda = '/'.join([kill, death, assist])
        img = item.select('div.ChampionImage img')[0].get('src')
        img_data = BytesIO(req.urlopen('https:' + img).read())
        
        im=Image.open(img_data)
        width, height = im.size
        
        print(width, height)

        # 엑셀 저장
        worksheet.write(f'A{row_num}', game_type)
        worksheet.write(f'B{row_num}', game_result)
        worksheet.write(f'C{row_num}', champion_name)
        worksheet.write(f'D{row_num}', kda)
        worksheet.insert_image(f'F{row_num}', champion_name, {'image_data':img_data, 'x_offset': 15, 'y_offset': 30})

        print(game_type, game_result, champion_name, kill, death, assist)

        row_num += 1
    
    print('가져오기 끝')
    browser.quit()
    workbook.close()

search(input('소환사 명을 입력하세요. : '))



ModuleNotFoundError: No module named 'PIL'